# End-to-End Demo

This notebook demonstrates the complete RAG-based microfinance eligibility assessment pipeline:
1. Load knowledge base
2. Load example user profile
3. Build profile from raw data
4. Run RAG assessment
5. Show results
6. Demonstrate consent flow


In [ ]:
# Install required packages
!pip install -q pdfplumber sentence-transformers faiss-cpu groq streamlit


In [ ]:
# Setup
import sys
from pathlib import Path
import os
import json

sys.path.append(str(Path.cwd()))

# Set Groq API key (get from https://console.groq.com/)
os.environ["GROQ_API_KEY"] = "YOUR_GROQ_API_KEY_HERE"  # Replace with your key

print("Setup complete!")


## Step 1: Load Example Profile

Load one of the example profiles from `example_inputs/` folder.


In [ ]:
# Load example profile
example_file = "example_inputs/likely_eligible_gig_worker.json"

with open(example_file, 'r') as f:
    raw_data = json.load(f)

print(f"Loaded profile: {raw_data['user_id']}")
print(f"Demographics: {raw_data['demographics']}")


## Step 2: Build Structured Profile

Convert raw data into the structured profile schema.


In [ ]:
# Build profile
from src.profile.profile_builder import build_profile

profile = build_profile(
    user_id=raw_data["user_id"],
    demographics=raw_data["demographics"],
    raw_unconventional_data=raw_data["_raw_unconventional"]
)

print("Profile built successfully!")
print(f"User ID: {profile['user_id']}")
print(f"Timestamp: {profile['timestamp']}")
print(f"\nMobile metadata: {profile['mobile_metadata']}")
print(f"\nPsychometrics: {profile['psychometrics']}")
print(f"\nFinancial behavior: {profile['financial_behavior']}")


## Step 3: Save Profile to Memory

Store the profile in local memory.


In [ ]:
# Save to memory
from src.memory.store import save_user_profile

save_user_profile(profile["user_id"], profile)
print(f"Profile saved to memory for user {profile['user_id']}")


## Step 4: Run RAG Assessment

Assess eligibility using the RAG agent.


In [ ]:
# Run RAG assessment
from src.agent.rag_agent import RAGAgent

agent = RAGAgent(
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model_name="mixtral-8x7b-32768"  # or "llama3-70b-8192", "gemma-7b-it"
)

decision = agent.assess_eligibility(profile, k=6)

print("Assessment complete!")
print(json.dumps(decision, indent=2))


## Step 5: Display Results

Show formatted results.


In [ ]:
# Display results
print("=" * 60)
print("ELIGIBILITY ASSESSMENT RESULTS")
print("=" * 60)
print(f"\nEligibility: {decision['eligibility'].upper()}")
print(f"Risk Score: {decision['risk_score']:.2f} (0.0 = lowest risk, 1.0 = highest risk)")
print(f"Confidence: {decision['confidence'].upper()}")
print(f"\nVerdict: {decision['verdict_text']}")

print("\n" + "-" * 60)
print("STRONG POINTS:")
for i, point in enumerate(decision['strong_points'], 1):
    print(f"  {i}. {point}")

print("\n" + "-" * 60)
print("WEAK POINTS:")
for i, point in enumerate(decision['weak_points'], 1):
    print(f"  {i}. {point}")

print("\n" + "-" * 60)
print("ACTIONABLE RECOMMENDATIONS:")
for i, rec in enumerate(decision['actionable_recommendations'], 1):
    print(f"  {i}. {rec}")

if decision['required_unconventional_data']:
    print("\n" + "-" * 60)
    print("REQUIRED ADDITIONAL DATA:")
    for item in decision['required_unconventional_data']:
        print(f"  - {item}")


## Step 6: Consent Flow

Demonstrate the consent mechanism. In a real application, this would be a user interaction.


In [ ]:
# Simulate consent flow
from src.memory.store import save_consented_submission, delete_user_data

# Simulate user consenting
user_consented = True  # In real app, this comes from UI

if user_consented:
    save_consented_submission(profile["user_id"], profile, decision)
    print(f"✅ User {profile['user_id']} consented. Data saved to bank database.")
else:
    delete_user_data(profile["user_id"])
    print(f"❌ User {profile['user_id']} declined. All data deleted.")


## Step 7: View Bank Database (Admin View)

View consented submissions (admin functionality).


In [ ]:
# View bank submissions
from src.memory.store import get_bank_submissions

submissions = get_bank_submissions(limit=10)

print(f"Total consented submissions: {len(submissions)}\n")

for i, sub in enumerate(submissions, 1):
    print(f"Submission {i}:")
    print(f"  User ID: {sub['user_id']}")
    print(f"  Timestamp: {sub['timestamp']}")
    print(f"  Eligibility: {sub['decision']['eligibility']}")
    print(f"  Risk Score: {sub['decision']['risk_score']}")
    print()
